In [ ]:
import py3dep
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pynhd as nhd
import flopy
import geopandas as gpd
from pynhd import NLDI, NHDPlusHR, WaterData
import contextily as ctx
import numpy as np

from flopy.utils import Raster
import os

import geopandas as gpd

In [ ]:
from xrspatial import zonal_stats
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
import rasterio
import xarray as xr

use xarray zonal stats to calculate minimum elelvation for grid cells


https://pygis.io/docs/e_raster_rasterize.html

In [ ]:
import basic

In [ ]:
m = basic.load_model()

In [ ]:
x = m.modelgrid.get_xcellcenters_for_layer(1)
y = m.modelgrid.get_ycellcenters_for_layer(1)


In [ ]:
mgrid = m.modelgrid

In [ ]:

dst_crs = 'EPSG:2226'
raster_ws = os.path.join("GIS", "DEM")
raster_name = "output_be_DTM.tif"
rast = os.path.join(raster_ws, raster_name)
rio = Raster.load(rast)



In [ ]:


arr = rio.get_array(1)
idx = np.isfinite(arr)

vmin, vmax = arr[idx].min(), arr[idx].max()
vmin, vmax



In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7);





In [ ]:
rio.bounds

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid)
pmv.plot_grid(ax=ax, lw=0.5, color="black");

In [ ]:
import cartopy.crs as ccrs

def coordXform(orig_crs, target_crs, x, y):
    return target_crs.transform_point(  x, y, orig_crs )

crs_longlat = ccrs.PlateCarree() 
crs_3857 = ccrs.epsg(2226)

coordXform(crs_longlat,crs_3857, start[1], start[0] )

In [ ]:
start = ( -122.876894, 38.492277)
stop = (  -122.876925, 38.494097)

start = coordXform(crs_longlat,crs_3857, start[0], start[1] )
stop = coordXform(crs_longlat,crs_3857, stop[0], stop[1] )

num_points = 90

# def line_slope_intercept(x1, y1, x2, y2):
#     """Calculate the slope and intercept of a line between two points."""
#     slope = (y2 - y1) / (x2 - x1)
#     intercept = y1 - slope * x1
#     return slope, intercept

def points_on_line(slope, intercept, start, end, num_points):
    """Generate equally spaced points on a line defined by its slope and intercept."""
    points = np.linspace(start, end, num_points)
    # y_values = slope * x_values + intercept
    # points = np.column_stack((x_values, y_values))
    
    return points

# slope, intercept = line_slope_intercept(start[0], start[1], stop[0], stop[1])
points = points_on_line(slope, intercept, start, stop, num_points)

In [ ]:
np.linspace(start, stop, num_points)

In [ ]:
points

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, aspect="equal")

ax = rio.plot(ax=ax, vmin=vmin, vmax=vmax)
plt.colorbar(ax.images[0], shrink=0.7)
pmv = flopy.plot.PlotMapView(modelgrid=m.modelgrid)
pmv.plot_grid(ax=ax, lw=0.5, color="black");

ax.scatter(start[0], start[1])
ax.scatter(stop[0], stop[1])

ax.scatter(points[:,0], points[:,1])


In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
rio.sample_point((6310688.157792784, 1942309.427433906))

In [ ]:
values = [rio.sample_point(p) for p in points]

In [ ]:
dist = squareform(pdist(points))[0]

In [ ]:
profile = np.vstack([dist, values]).T

In [ ]:
plt.scatter(profile[:,0],profile[:,1])
n = 34
# locs = np.arange(n,n+9*2,2)
locs = [ 34, 39, 40, 41, 42,44, 46, 52]
plt.scatter(profile[:,0][locs],profile[:,1][locs])

df = pd.DataFrame(profile[locs,:], columns = ['distance','height'])

df.loc[:,'distance'] = df.loc[:,'distance']-df.loc[:,'distance'].min()
df.loc[:,'height'] = df.loc[:,'height']-df.loc[:,'height'].min()
df.to_csv(r'GIS\Stream_GEO\markwest_creek_xsection.csv')